In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

Check the eigenvalues of connectivity matrix and overlap matrix are the same, but the singular values of these two are different

In [2]:
JE,JI,a,b=1.2,0.8,0.1,0.1#15151515151505
print('JE,JI,a,b:',JE,JI,a,b)
# calculate SVD
Jt = np.zeros((2,2))
Jt[:,0],Jt[:,1]=JE,-JI
Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
# Jt[1,0]=Jt[0,1]
lvec,sv,rvech=la.svd(Jt)
m=lvec.copy()
m[:,0],m[:,1]=m[:,0]*sv[0],m[:,1]*sv[1]
n=rvech.T
M,N=np.array([[1,1],[1,-1]]),np.array([[JE,-a],[-JI,b]])
# assume the diagonal derivative matrix is
diagphi=np.zeros((2,2))
diagphi[0,0],diagphi[1,1]=0.5,0.8

# the connectivity matrix
connmat=m@n.T@diagphi
connmatf=diagphi@m@n.T

overlapmat=n.T@diagphi@m
eigvc,_=la.eig(connmat)
eigvcf,_=la.eig(connmatf)
eigvo,_=la.eig(overlapmat)
overMN=N.T@diagphi@M
eigvoM,_=la.eig(overMN)
print('eigenvalue of connectivity matrix:',eigvc,' designed:',eigvcf,' overlap matrix:',eigvo,' overlap matrix under M:',eigvoM)

_,svc,_=la.svd(connmat)
_,svcf,_=la.svd(connmatf)
_,svo,_=la.svd(overlapmat)
_,svoM,_=la.svd(overMN)
print('singular value of connectivity matrix:',svc,' designed:',svcf,' overlap matrix:',svo,' overlap matrix under M:',svoM)

JE,JI,a,b: 1.2 0.8 0.1 0.1
eigenvalue of connectivity matrix: [ 0.11305302 -0.28305302]  designed: [ 0.11305302 -0.28305302]  overlap matrix: [-0.28305302  0.11305302]  overlap matrix under M: [ 0.11305302 -0.28305302]
singular value of connectivity matrix: [1.24753439 0.0256506 ]  designed: [1.42284721 0.02249012]  overlap matrix: [1.42284721 0.02249012]  overlap matrix under M: [1.24753439 0.0256506 ]


* D-transform, as a function of z (z is the limit of singular value for perturbed matrix). varphi-transform, as a function of z (also \rho).
* Critical/threshold theta

In [3]:
# Functions for D-transform, varphi-transform, inverse D-transform
def D_transform(x,*data):
    theta_i,c=data
    if c==1:
        x0=float(x[0])
        am=(x0**2-2-np.sqrt((2-x0**2)**2-4))/2.0
        # print('am:',am)
        return[(x0*am/(am+1))**2-1/theta_i**2]

# derivative of the D-transform
def derivD_transform(rho_limit):
    # am=(rho_limit**2-2-np.sqrt((2-rho_limit**2)**2-4))/2.0
    # coeff=2*(am/(am+1))**2
    epsp=1e-5
    x0=float(rho_limit)
    am=(x0**2-2-np.sqrt((2-x0**2)**2-4))/2.0
    orgD_transform=(x0*am/(am+1))**2
    # perturb
    x0+=epsp
    am=(x0**2-2-np.sqrt((2-x0**2)**2-4))/2.0
    print('am',am)
    pD_transform=(x0*am/(am+1))**2
    derivDtrans=(pD_transform-orgD_transform)/epsp

    return derivDtrans

# varphi transform 
def varphi_transform(rho_limit):
    am=(rho_limit**2-2-np.sqrt((2-rho_limit**2)**2-4))/2.0
    return am*rho_limit/(am+1)

# vector projection
def vec_projection(limitrho,sv,theta):
    # find out the outliers, which are larger than the critical theta
    thetac,supremum=theta
    thetai=sv[np.where(sv>thetac)]
    # print('size of the outliers:',len(thetai))
    # there are a list of theta, which are in the phase transition region
    rho_limits = np.zeros(len(thetai))
    varphis,derivDtransforms=np.zeros(len(thetai)),np.zeros(len(thetai))
    lvecproj,rvecproj=np.zeros_like(rho_limits),np.zeros_like(rho_limits)
    for idx, thetai0 in enumerate(thetai):
        # data=(thetai0,1)
        # x0=supremum*1.2
        # rho_limits[idx]=fsolve(D_transform,x0,data) # recalculate limit rho
        rho_limits[idx]=limitrho[idx]
        varphis[idx]=varphi_transform(rho_limits[idx])
        derivDtransforms[idx]=derivD_transform(rho_limits[idx])

        lvecproj[idx]=(1-sv[idx]**(-2))#-2*varphis[idx]/thetai0**2/derivDtransforms[idx]
        rvecproj[idx]=-2*varphis[idx]/thetai0**2/derivDtransforms[idx]
    return (lvecproj,rvecproj)
        




# # check the D-transform
# svlr=2.82842712
# data=(svlr,1)
# x0=svlr
# svperturb=fsolve(D_transform,x0,data)
# print('limit singular value:',svperturb)



In [4]:
# generate perturbed matrix
nn=600
JE,JI=0.50,0.502
a,b=0.0,0.0#0.80,0.2#1.5,-0.3
# a=-JE*b/JI
# a=-JE*JI/b
# a=-12.0
print('a now:',a)
J=np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn

lvec,sv,rvech=la.svd(J)
rvec=rvech.T
print('singular value:',sv[0:2])
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_=x+J
lvec_,sv_,rvech_=la.svd(x_)
thetac = 1.0 # critical value for low rank
supremumsv=2.0 # supremum of distribution of the singular values (random matrix)
print('singularvalue:',sv_[:3])
rvec_=rvech_.T
outlier=sv_[np.where(sv_>supremumsv)]

# find outliers
svi=sv[np.where(sv>thetac)]
theo_outlier,lvecproj,rvecproj=np.zeros(len(svi)),np.zeros(len(svi)),np.zeros(len(svi))
for idx, svi0 in enumerate(svi):
    svlr=svi0
    data=(svlr,1)
    x0=sv_[idx]
    theo_outlier[idx]=fsolve(D_transform,x0,data)
    lvecproj[idx],rvecproj[idx]=np.sum(lvec[:,idx]*lvec_[:,idx]),np.sum(rvec[:,idx]*rvec_[:,idx])
print('compare num and theo outlier:',outlier,'; ',theo_outlier)

theo_lvecproj,theo_rvecproj=vec_projection(theo_outlier,sv,theta=[thetac,supremumsv])
print('compare n um and theo vector projection:')
print('projection of left singular vector:',lvecproj**2,'; ',theo_lvecproj)
print('projection of right singular vector:',rvecproj**2,'; ',theo_rvecproj)


a now: 0.0
singular value: [1.00200200e+00 7.82531421e-15]
singularvalue: [2.01784177 1.99717104 1.98008905]
compare num and theo outlier: [2.01784177] ;  [2.01784177]
am 0.7656557257983869
compare n um and theo vector projection:
projection of left singular vector: [0.19517171] ;  [0.003992]
projection of right singular vector: [0.19915678] ;  [0.30480532]


Fix three parameters $J_E,\ J_I,\ b$, change $a$, turning point $a=-J_Eb/J_I$ relates to the minimum $\sigma_1$, turning point $a=-J_EJ_I/b$ relates to the maximum $\sigma_2$

In [5]:
# generate perturbed matrix
nn=600
JE,JI=0.5,0.3
a,b=0.80,0.25#1.5,-0.3
a1=-JI*b/JE
a2=-JE*JI/b
print('a1-',a1,' a2-',a2)
# b1=-JE*a/JI
# b2=-JE*JI/a
# print('b1-',b1,' b2-',b2)
aset=np.linspace(-3.0,0.6,50)
sigma_2=np.zeros((2,len(aset)))
for idx, a in enumerate(aset):
    J=np.zeros((nn*2,nn*2))
    J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
    J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
    J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn

    lvec,sv,rvech=la.svd(J)
    sigma_2[:,idx]=sv[:2]

fig = plt.figure(figsize=(8,6))
ax1 = fig.add_subplot(111)
ax1.plot(aset,sigma_2[0,:],'r',label=r'$\sigma_1$')
ax1.plot(aset,sigma_2[1,:],'b',label=r'$\sigma_2$')
ax1.set_title('JE='+np.str(JE)+',JI='+np.str(JI)+',b='+np.str(b))
ax1.set_label(r'$a\in \left [-3.0,0.6 \right ]$')
ax1.set_ylabel(r'$\sigma$')
plt.legend()
amin=aset[np.where(sigma_2[0,:]==np.min(sigma_2[0,:]))]
print('aminimum:',amin)
# plt.show()
# # idxpos=np.where(lvec_[:,0]>0)
# # idxneg=np.where(lvec_[:,0]<0)
# # print('  pos:',np.shape(idxpos),'; number neg:',np.shape(idxneg))
# # print(lvec_[:6,0],' and ',lvec_[299:305,0])

a1- -0.15  a2- -0.6
aminimum: [-0.13469388]


In [6]:
ax1.plot(aset,np.ones_like(aset),'k--')
ax1.axis('square')
print(amin,JI*b/JE)
amax2=aset[np.where(sigma_2[1,:]==np.max(sigma_2[1,:]))]
print('amax for 2:',amax2)
print(amax2,JI*JE/b)

[-0.13469388] 0.15
amax for 2: [-0.5755102]
[-0.5755102] 0.6


In [7]:
a,b=0.2,0.2#1.5,-0.3
# a1=-JE*b/JI
# a2=-JE*JI/b
# print('a1-',a1,' a2-',a2)
b1=-JI*a/JE
b2=-JE*JI/a
print('b1-',b1,' b2-',b2)
bset=np.linspace(-3.0,0.6,50)
sigma_2b=np.zeros((2,len(bset)))
for idx, b in enumerate(bset):
    J=np.zeros((nn*2,nn*2))
    J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
    J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
    J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn

    lvec,sv,rvech=la.svd(J)
    sigma_2b[:,idx]=sv[:2]

# fig = plt.figure(figsize=(8,6))
ax2 = fig.add_subplot(122)
ax2.plot(bset,sigma_2b[0,:],'r',label=r'$\sigma_1$')
ax2.plot(bset,sigma_2b[1,:],'b',label=r'$\sigma_1$')
plt.legend()

b1- -0.12  b2- -0.7499999999999999


Testing the whether the singular values of 2by2 matrix differ from those of NbyN matrix.
Testing both the rank-1 and rank-2 cases
Finished

In [8]:
# generate 2by2 matrix
nn=1
JE,JI=2.5,1.3
a,b=0.0,0.0#1.5,0.25#0.3,0.2#
# a=-JI*(JE/b)#+0.8
# a=-JI/(JE/b)#-0.3
print('a', a)
J=np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn
print('2by2 matrix:',J)
lvec,sv,rvech=la.svd(J)
print('sum:',np.sum(sv),'diff:',sv[0]-sv[1])
s1=np.sqrt(2*((JE+b)**2+(JI-a)**2))
s2=np.sqrt(2*((JE-b)**2+(JI+a)**2))
print('sum1:',s1,'; sum2:',s2)
# generate perturbed matrix
nn=600
# # JE,JI=0.8,0.6
# # a,b=0.80,0.2#1.5,-0.3
# a1=-JE*b/JI
# a2=-JE*JI/b
# print('a1-',a1,' a2-',a2)
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print('2by2 singular value:',sv[:],'; random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_=x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]
print('theoretical solution:',theo_sv,'; numerical solution:',sv_[:2])

a 0.0
2by2 matrix: [[ 2.5 -1.3]
 [ 2.5 -1.3]]
sum: 3.984971769034255 diff: 3.984971769034255
sum1: 3.984971769034255 ; sum2: 3.984971769034255
2by2 singular value: [3.98497177e+00 1.71892481e-16] ; random singular value: [3.98497177e+00 4.79417906e-14]
theoretical solution: [4.23591458e+00 2.08586285e+13] ; numerical solution: [4.24607762 1.9887494 ]


In [9]:
# 2910 reconfinement
nn=1
JE,JI=1.60,1.2
a,b=0.0,0.0#1.5,0.25#0.3,0.2#
# a=-JI*(JE/b)#+0.8
# a=-JI/(JE/b)#-0.3
print('a', a)
J=np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/nn,-JI/nn
J[:nn,:nn],J[nn:,:nn]=J[:nn,:nn]-a/nn,J[nn:,:nn]+a/nn
J[:nn,nn:],J[nn:,nn:]=J[:nn,nn:]+b/nn,J[nn:,nn:]-b/nn
print('2by2 matrix:',J)
lvec,sv,rvech=la.svd(J)
print('sum:',np.sum(sv),'diff:',sv[0]-sv[1])
s1=np.sqrt(2*((JE+b)**2+(JI-a)**2))
s2=np.sqrt(2*((JE-b)**2+(JI+a)**2))
print('sum1:',s1,'; sum2:',s2)
# generate perturbed matrix
nn=600
# # JE,JI=0.8,0.6
# # a,b=0.80,0.2#1.5,-0.3
# a1=-JE*b/JI
# a2=-JE*JI/b
# print('a1-',a1,' a2-',a2)
Jr=np.zeros((nn*2,nn*2))
Jr[:,:nn],Jr[:,nn:]=JE/nn,-JI/nn
Jr[:nn,:nn],Jr[nn:,:nn]=Jr[:nn,:nn]-a/nn,Jr[nn:,:nn]+a/nn
Jr[:nn,nn:],Jr[nn:,nn:]=Jr[:nn,nn:]+b/nn,Jr[nn:,nn:]-b/nn
lvec,svr,rvech=la.svd(Jr)
print('2by2 singular value:',sv[:],'; random singular value:',svr[:2])

# add random matrix
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
x_=x+Jr
lvec_,sv_,rvech_=la.svd(x_)
theo_sv=1/svr[:2]+svr[:2]
print('theoretical solution:',theo_sv,'; numerical solution:',sv_[:2])

a 0.0
2by2 matrix: [[ 1.6 -1.2]
 [ 1.6 -1.2]]
sum: 2.8284271247461907 diff: 2.8284271247461907
sum1: 2.8284271247461903 ; sum2: 2.8284271247461903
2by2 singular value: [2.82842712e+00 1.81513847e-17] ; random singular value: [2.82842712e+00 2.51212613e-14]
theoretical solution: [3.18198052e+00 3.98069184e+13] ; numerical solution: [3.19039409 1.99278233]


In [10]:
rholimit=1/sv[0]+sv[0]
derivphi=1/(1-sv[0]**2)
overlap=-1/derivphi/sv[0]**2
print('am theo:',1/sv[0]**2)
print('overlap:',overlap)
print('overlap num:',np.sum(lvec_[:,0]*lvec[:,0])**2)

## calculate whether there is an additive relationship
overlap_num=1.0-1/sv[0]**2
alpha = np.sqrt(overlap_num)
lvar=lvec_[:,0]-alpha*lvec[:,0]
# plt.figure()
# n, bins, patches = plt.hist(lvar[:nn], 100, density=True, facecolor='g', alpha=0.75)

rvec_,rvec=rvech_.T,rvech.T
rvar=rvec_[:,0]-alpha*rvec[:,0]
# plt.figure()
# n, bins, patches = plt.hist(rvar[:nn], 100, density=True, facecolor='g', alpha=0.75)

#tilte overlap
tilteuv=np.sum(rvec_[:,0]*lvec_[:,0])
orguv=alpha**2*np.sum(rvec[:,0]*lvec[:,0])
cosuv=(1/sv[0]**2)*np.sum(rvec[:,0]*lvec[:,0])
print(cosuv)
print('Gaussian relationship, org:',orguv,', tilte:',tilteuv)
diffuv=(tilteuv-orguv)
print('Diff uv:',diffuv)

# Tomorrow, check whether the overlap of the perturbed matrix is equal to the original * alpha square.

am theo: 0.12499999999999994
overlap: 0.875
overlap num: 0.877267788193405
0.01767766952966339
Gaussian relationship, org: 0.12374368670764377 , tilte: 0.12076751875989811
Diff uv: -0.0029761679477456615


Find the limit of singular value of perturbed matrix, by finding the intersection between lines $y=x$ and $\frac{1}{\rho}\left (1+\frac{2}{\rho^2-2-\sqrt{\left (2-\rho^2\right )^2-4}}\right )$

In [11]:
# Plot the intersections
# when low-rank singular value is larger than the threshold value \bar\theta=1, there is a phase transition 
xtheta=np.linspace(0.8,6.8,100)
theta_i=2.8284271247*np.ones_like(xtheta)
# l.h.s function
ylhs=theta_i
# r.h.s function 
yrhs=1/xtheta*(1+2/(xtheta**2-2-np.sqrt((2-xtheta**2)**2-4)))
fig=plt.figure()
ax0=fig.add_subplot(111)
ax0.plot(xtheta,ylhs,'r',label='low-rank singular value')
ax0.plot(xtheta,yrhs,'b',label='limit singular value')
plt.legend()


Inspired by Ljubicia's presentation
* what's the difference between R(u=0) and Gaussian Random Network
* is there a phase transition when using a rank-1 gaussian structure?
* test it!
